In [1]:
# import the necessary packages
import argparse
import datetime
import time
import cv2
import pandas as pd

from imutils.video import videostream
import imutils


DATA_FILE = 'camera_files.csv'

file_df = pd.read_csv(DATA_FILE)

print(file_df)


                                                  file  human  cat  squirrel
0    E:\cam\2020\12\20\Under Patio_01_2020122023580...      1    0         0
1    E:\cam\2020\12\20\Under Patio_01_2020122015320...      1    0         0
2    E:\cam\2020\12\20\Under Patio_01_2020122015491...      1    0         0
3    E:\cam\2020\12\20\Under Patio_01_2020122016115...      1    0         0
4    E:\cam\2020\12\20\Under Patio_01_2020122016273...      0    0         0
..                                                 ...    ...  ...       ...
249  E:\cam\2020\12\31\Under Patio_01_2020123111232...      1    0         0
250  E:\cam\2020\12\31\Under Patio_01_2020123111262...      1    0         0
251  E:\cam\2020\12\31\Under Patio_01_2020123115253...      0    1         0
252  E:\cam\2020\12\31\Under Patio_01_2020123116553...      0    0         0
253  E:\cam\2020\12\31\Under Patio_01_2020123120413...      0    0         0

[254 rows x 4 columns]


In [12]:
file_data = file_df.iloc[1]
print(file_data["file"])
print(file_data)

vs = cv2.VideoCapture(file_data["file"])

update_period = 1
min_area = 500
skip_frames = 60

# initialize the first frame in the video stream
firstFrame = None

# while(vs.isOpened()):
#     ret, frame = vs.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     cv2.imshow('frame',gray)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# vs.release()
# cv2.destroyAllWindows()

last_clear = 0

# loop over the frames of the video
while True:

    #https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/

    # grab the current frame and initialize the occupied/unoccupied
    # text
    ret, frame = vs.read()
    #frame = frame if args.get("video", None) is None else frame[1]
    text = "Unoccupied"
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if not ret or frame is None:
        break
    if skip_frames > 0:
        skip_frames -= 1
        continue
    # resize the frame, convert it to grayscale, and blur it
    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    # if the first frame is None, initialize it
    if firstFrame is None or last_clear + update_period < time.time():
        firstFrame = gray
        last_clear = time.time()
        print('resetting BG')
        continue
    # compute the absolute difference between the current frame and
    # first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]
    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < min_area:
            continue
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = "Occupied"
    # draw the text and timestamp on the frame
    cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
        (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
    # show the frame and record if the user presses a key
    cv2.imshow("Security Feed", frame)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key is pressed, break from the lop
    if key == ord("q"):
        break
# cleanup the camera and close any open windows
vs.release()
cv2.destroyAllWindows()


E:\cam\2020\12\20\Under Patio_01_20201220153209.mp4
file        E:\cam\2020\12\20\Under Patio_01_2020122015320...
human                                                       1
cat                                                         0
squirrel                                                    0
Name: 1, dtype: object
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
resetting BG
r